In [93]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import csv
from statistics import median

import surprise
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

In [94]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [95]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

In [96]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
user_and_item_data71mb = []

f = gzip.open("user_and_item_data71mb.json.gz")
for l in f:
    d = eval(l)
    user_and_item_data71mb.append(d)
    if len(user_and_item_data71mb) >= 80000:
        break
        
f.close()

In [98]:
review_data_6mb = []

f = gzip.open("review_data_6mb.json.gz")
for l in f:
    d = eval(l)
    review_data_6mb.append(d)
    if len(review_data_6mb) >= 50000:
        break
        
f.close()

In [99]:
review_data_6mb[4]

{'user_id': 'maplemage',
 'user_url': 'http://steamcommunity.com/id/maplemage',
 'reviews': [{'funny': '3 people found this review funny',
   'posted': 'Posted April 15, 2014.',
   'last_edited': '',
   'item_id': '211420',
   'helpful': '35 of 43 people (81%) found this review helpful',
   'recommend': True,
   'review': 'Git gud'},
  {'funny': '1 person found this review funny',
   'posted': 'Posted December 23, 2013.',
   'last_edited': '',
   'item_id': '211820',
   'helpful': '12 of 16 people (75%) found this review helpful',
   'recommend': True,
   'review': "It's like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."},
  {'funny': '2 people found this review funny',
   'posted': 'Posted March 14, 2014.',
   'last_edited': '',
   'item_id': '730',
   'helpful': '5 of 5 people (100%) found this review helpful',
   'recommend': True,
   'review': 'Hold shift to win, Hold CTRL to lose.'},
  {'funny': '',
   'posted': 'Posted July 

In [100]:
user_and_item_data71mb[5]

{'user_id': 'MinxIsBetterThanPotatoes',
 'items_count': 371,
 'steam_id': '76561198004744620',
 'user_url': 'http://steamcommunity.com/id/MinxIsBetterThanPotatoes',
 'items': [{'item_id': '50',
   'item_name': 'Half-Life: Opposing Force',
   'playtime_forever': 256,
   'playtime_2weeks': 0},
  {'item_id': '240',
   'item_name': 'Counter-Strike: Source',
   'playtime_forever': 167,
   'playtime_2weeks': 0},
  {'item_id': '320',
   'item_name': 'Half-Life 2: Deathmatch',
   'playtime_forever': 3674,
   'playtime_2weeks': 0},
  {'item_id': '4000',
   'item_name': "Garry's Mod",
   'playtime_forever': 358,
   'playtime_2weeks': 0},
  {'item_id': '6570',
   'item_name': 'Onimusha 3: Demon Siege',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '220',
   'item_name': 'Half-Life 2',
   'playtime_forever': 1299,
   'playtime_2weeks': 0},
  {'item_id': '340',
   'item_name': 'Half-Life 2: Lost Coast',
   'playtime_forever': 29,
   'playtime_2weeks': 0},
  {'item_id': '360',
  

In [101]:
len(user_and_item_data71mb)

80000

In [102]:
len(review_data_6mb)

25799

In [103]:
item_metadata = []

f = gzip.open("item_metadata.json.gz")
for l in f:
    d = eval(l)
    item_metadata.append(d)
    if len(item_metadata) >= 35000:
        break
        
f.close()

In [104]:
len(item_metadata)

32135

In [105]:
def merge_datasets_with_matching(review_data, user_and_item_data):
    """
    Merge datasets by matching user IDs with extensive debugging
    
    :param review_data: List of user review data
    :param user_and_item_data: List of user item and playtime data
    :return: List of merged data entries
    """
    # Verify input dataset lengths
    print(f"Original Review Data Length: {len(review_data)}")
    print(f"Original Items Data Length: {len(user_and_item_data)}")
    
    # Create lookup dictionaries for efficient matching
    review_dict = {entry['user_id']: entry for entry in review_data}
    items_dict = {entry['user_id']: entry for entry in user_and_item_data}
    
    # Detailed user ID analysis
    review_user_ids = set(review_dict.keys())
    items_user_ids = set(items_dict.keys())
    
    print(f"Unique User IDs in Review Data: {len(review_user_ids)}")
    print(f"Unique User IDs in Items Data: {len(items_user_ids)}")
    
    # Find common user IDs
    common_user_ids = review_user_ids & items_user_ids
    
    # More detailed tracking
    unmatched_review_users = review_user_ids - items_user_ids
    unmatched_items_users = items_user_ids - review_user_ids
    
    print(f"Common User IDs: {len(common_user_ids)}")
    print(f"Unmatched Users in Review Data: {len(unmatched_review_users)}")
    print(f"Unmatched Users in Items Data: {len(unmatched_items_users)}")
    
    # Sample of unmatched user IDs for investigation
    print("\nSample of Unmatched Review User IDs:")
    print(list(unmatched_review_users)[:10])
    print("\nSample of Unmatched Items User IDs:")
    print(list(unmatched_items_users)[:10])
    
    # Store merged data
    merged_data = []
    
    # Tracking variables
    total_reviews_processed = 0
    total_merged_entries = 0
    
    # Process only users present in both datasets
    for user_id in common_user_ids:
        user_reviews = review_dict[user_id]['reviews']
        user_items = items_dict[user_id]['items']
        
        total_reviews_processed += len(user_reviews)
        
        # Match reviews with items
        for review in user_reviews:
            # Find matching items
            matching_items = [
                item for item in user_items 
                if str(item['item_id']) == str(review['item_id'])
            ]
            
            # Create merged entries for matching items
            for item in matching_items:
                merged_entry = {
                    'user_id': user_id,
                    'item_id': review['item_id'],
                    'review_text': review.get('review', ''),
                    'recommend': review.get('recommend'),
                    'playtime_forever': item.get('playtime_forever', 0),
                    'playtime_2weeks': item.get('playtime_2weeks', 0)
                }
                merged_data.append(merged_entry)
                total_merged_entries += 1
    
    # Comprehensive summary
    print("\nFinal Matching Summary:")
    print(f"Total Users in Review Data: {len(review_dict)}")
    print(f"Total Users in Items Data: {len(items_dict)}")
    print(f"Common Users: {len(common_user_ids)}")
    print(f"Total Reviews Processed: {total_reviews_processed}")
    print(f"Total Merged Entries: {total_merged_entries}")
    
    return merged_data

In [106]:
merged_data = merge_datasets_with_matching(review_data_6mb,user_and_item_data71mb)

Original Review Data Length: 25799
Original Items Data Length: 80000
Unique User IDs in Review Data: 25485
Unique User IDs in Items Data: 79316
Common User IDs: 23925
Unmatched Users in Review Data: 1560
Unmatched Users in Items Data: 55391

Sample of Unmatched Review User IDs:
['76561198097851250', '76561198100899460', 'Salttyy', 'mephistocrates', 'myeki', 'OpaopapaGamer', '76561198093272625', '76561198095448607', '76561198097798959', '76561198116458543']

Sample of Unmatched Items User IDs:
['joshdenn', '76561198006450473', 'whazzuup', '76561198073313711', '76561198054776845', '76561198065053986', '76561198074678155', '76561198046980884', '76561198067068112', 'joeyhiro']

Final Matching Summary:
Total Users in Review Data: 25485
Total Users in Items Data: 79316
Common Users: 23925
Total Reviews Processed: 55413
Total Merged Entries: 42210


In [107]:
merged_data[8]

{'user_id': 'Buhmann',
 'item_id': '49520',
 'review_text': 'Jogo louco, curti muito!',
 'recommend': True,
 'playtime_forever': 7529,
 'playtime_2weeks': 318}

In [108]:
len(merged_data)

42210

In [109]:
unique_user_ids = len(set(entry['user_id'] for entry in merged_data))
print(f"Number of unique user IDs: {unique_user_ids}")

# To get number of unique item IDs
unique_item_ids = len(set(entry['item_id'] for entry in merged_data))
print(f"Number of unique item IDs: {unique_item_ids}")

# If you want to see both at once
print("Unique user IDs:", len(set(entry['user_id'] for entry in merged_data)))
print("Unique item IDs:", len(set(entry['item_id'] for entry in merged_data)))

Number of unique user IDs: 18866
Number of unique item IDs: 3054
Unique user IDs: 18866
Unique item IDs: 3054


In [110]:
# Count recommendations
recommendations = [entry['recommend'] for entry in merged_data]

# Using numpy for a quick count
import numpy as np
unique, counts = np.unique(recommendations, return_counts=True)
print("Recommendation Distribution:")
print(dict(zip(unique, counts)))

# Alternatively, using a simple count method
true_count = recommendations.count(True)
false_count = recommendations.count(False)

print("\nRecommend True Count:", true_count)
print("Recommend False Count:", false_count)
print("Percentage of Recommendations:", true_count / len(recommendations) * 100, "%")

Recommendation Distribution:
{False: 4753, True: 37457}

Recommend True Count: 37457
Recommend False Count: 4753
Percentage of Recommendations: 88.7396351575456 %


In [111]:
import random

# Separate recommended and non-recommended entries
recommend_true = [entry for entry in merged_data if entry['recommend'] == True]
recommend_false = [entry for entry in merged_data if entry['recommend'] == False]

# Calculate target number of True recommendations (about 66%)
total_desired = len(recommend_false) * 1  # keep all False entries, add same number of True

# Randomly sample from True recommendations
balanced_recommend_true = random.sample(recommend_true, total_desired)

# Recombine the datasets
balanced_merged_data = recommend_false + balanced_recommend_true

# Shuffle the balanced dataset
random.shuffle(balanced_merged_data)

# Verify new distribution
recommendations = [entry['recommend'] for entry in balanced_merged_data]
unique, counts = np.unique(recommendations, return_counts=True)
print("New Recommendation Distribution:")
print(dict(zip(unique, counts)))

New Recommendation Distribution:
{False: 4753, True: 4753}


In [112]:
balanced_merged_data[0]

{'user_id': 'insertnameherere',
 'item_id': '65980',
 'review_text': "less is more?, not really(let me just say that civ 5 was my first civ, so i didn't have any previous civ titles to put it in context, but know when you compard BE to 5 its like its DLC, but also taking things out)i feel like they changed the things that worked, and kept the things i couldn't care for. like the tech web, your pretty much forced to go by the inner ring first, then the second and so forth, it looked more impressive in civ 5 scrolling all the way to the end of the tech tree and seeing how much progress was to be made. The religions gone, which im alright with because it was kinda hard to control and influence anyway, but less is more?The units, while the idea of upgrading them along a specifc line is cool and kind of felt like x-com, the actually outcome of this is that they look slightley different each time and don't really do anything special, you can't have that civ 5 x2 attack, +1 range super units 

In [113]:
len(balanced_merged_data)

9506

In [114]:
# Ensure valid entries in balanced_merged_data and item_metadata
balanced_merged_data = [entry for entry in balanced_merged_data if 'item_id' in entry]
item_metadata = [entry for entry in item_metadata if 'id' in entry]

# Extract all item_ids from balanced_merged_data into a set for fast lookup
valid_item_ids = {entry['item_id'] for entry in balanced_merged_data}

# Filter the item_metadata list based on the presence of the 'id' in valid_item_ids
cleaned_item_metadata = [entry for entry in item_metadata if entry['id'] in valid_item_ids]

# Print the cleaned list size (optional)
print(f"Cleaned item_metadata contains {len(cleaned_item_metadata)} entries.")

Cleaned item_metadata contains 1520 entries.


In [115]:
# Extract all valid item names
valid_item_names = {
    item['item_name']
    for user in user_and_item_data71mb
    for item in user['items']
}

# Filter item_metadata with .get() for safe key access
cleaned_item_metadata = [
    entry for entry in item_metadata
    if entry.get('app_name') in valid_item_names or entry.get('title') in valid_item_names
]

# Print results
print(f"Cleaned item_metadata contains {len(cleaned_item_metadata)} entries.")

Cleaned item_metadata contains 7846 entries.


In [116]:
# Step 1: Extract all valid item_ids from balanced_merged_data into a set
valid_item_ids = {entry['item_id'] for entry in merged_data}

# Step 2: Filter cleaned_item_metadata based on presence of 'id' in valid_item_ids
cleaned_item_metadata_further = [
    entry for entry in cleaned_item_metadata
    if entry['id'] in valid_item_ids  # Direct comparison since both are strings
]

# Step 3: Print results
print(f"Further cleaned item_metadata contains {len(cleaned_item_metadata_further)} entries.")

Further cleaned item_metadata contains 2163 entries.


In [117]:
# Step 1: Extract all valid item_ids from balanced_merged_data into a set
valid_item_ids = {entry['item_id'] for entry in balanced_merged_data}

# Step 2: Filter cleaned_item_metadata based on presence of 'id' in valid_item_ids
cleaned_item_metadata_further = [
    entry for entry in cleaned_item_metadata
    if entry['id'] in valid_item_ids  # Direct comparison since both are strings
]

# Step 3: Print results
print(f"Further cleaned item_metadata contains {len(cleaned_item_metadata_further)} entries.")

Further cleaned item_metadata contains 1189 entries.


In [118]:
len(balanced_merged_data)

9506

In [119]:
len(cleaned_item_metadata_further)

1189

In [120]:
cleaned_item_metadata_further[0]

{'publisher': 'Valve',
 'genres': ['Action'],
 'app_name': 'Half-Life',
 'sentiment': 'Overwhelmingly Positive',
 'title': 'Half-Life',
 'url': 'http://store.steampowered.com/app/70/HalfLife/',
 'release_date': '1998-11-08',
 'tags': ['FPS',
  'Classic',
  'Action',
  'Sci-fi',
  'Singleplayer',
  'Shooter',
  'Aliens',
  'First-Person',
  'Multiplayer',
  "1990's",
  'Story Rich',
  'Adventure',
  'Atmospheric',
  'Silent Protagonist',
  'Great Soundtrack',
  'Moddable',
  'Linear',
  'Retro',
  'Difficult',
  'Funny'],
 'reviews_url': 'http://steamcommunity.com/app/70/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player', 'Multi-player', 'Valve Anti-Cheat enabled'],
 'price': 9.99,
 'metascore': 96,
 'early_access': False,
 'id': '70',
 'developer': 'Valve'}

In [121]:
balanced_merged_data[0]

{'user_id': 'insertnameherere',
 'item_id': '65980',
 'review_text': "less is more?, not really(let me just say that civ 5 was my first civ, so i didn't have any previous civ titles to put it in context, but know when you compard BE to 5 its like its DLC, but also taking things out)i feel like they changed the things that worked, and kept the things i couldn't care for. like the tech web, your pretty much forced to go by the inner ring first, then the second and so forth, it looked more impressive in civ 5 scrolling all the way to the end of the tech tree and seeing how much progress was to be made. The religions gone, which im alright with because it was kinda hard to control and influence anyway, but less is more?The units, while the idea of upgrading them along a specifc line is cool and kind of felt like x-com, the actually outcome of this is that they look slightley different each time and don't really do anything special, you can't have that civ 5 x2 attack, +1 range super units 

In [122]:
# Step 1: Search for the game with id '4000'
game_name = None

for entry in cleaned_item_metadata_further:
    if entry['id'] == '4000':  # Match the id
        game_name = entry['app_name']  # or entry['title'] depending on your preference
        break  # Exit loop once the game is found

# Step 2: Print the result
if game_name:
    print(f"Game name for id '4000': {game_name}")
else:
    print("Game with id '4000' not found.")

Game name for id '4000': Garry's Mod


In [123]:
X = [[entry['playtime_forever'] + entry['playtime_forever'],len(entry['review_text']),  # number of question marks
      1]  # bias term (constant feature)
     for entry in balanced_merged_data]

y = [entry['recommend'] for entry in balanced_merged_data]

In [124]:
len(X)

9506

In [125]:
unique_user_ids = len(set(entry['user_id'] for entry in balanced_merged_data))
print(f"Number of unique user IDs: {unique_user_ids}")

# To get number of unique item IDs
unique_item_ids = len(set(entry['item_id'] for entry in balanced_merged_data))
print(f"Number of unique item IDs: {unique_item_ids}")

# If you want to see both at once
print("Unique user IDs:", len(set(entry['user_id'] for entry in balanced_merged_data)))
print("Unique item IDs:", len(set(entry['item_id'] for entry in balanced_merged_data)))

Number of unique user IDs: 6915
Number of unique item IDs: 1738
Unique user IDs: 6915
Unique item IDs: 1738


### setting len(recommend_false) * 1 and no weght_class balanced gives slighly better performance

In [126]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,shuffle=True)

# Train logistic regression
mod = LogisticRegression(C=1)
mod.fit(X_train, y_train)

# Evaluate
#print("Model Score:", mod.score(X_test, y_test))
#print("Coefficients:", mod.coef_)
predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test
print(sum(correct) / len(correct))

0.5986537652503156


In [127]:
# Random Baseline Classifier
baseline = DummyClassifier(strategy='stratified', random_state=42)
baseline.fit(X_train, y_train)
baseline_pred = baseline.predict(X_test)

print("Baseline (Random) Accuracy:", accuracy_score(y_test, baseline_pred))
print("\nBaseline Classification Report:")
print(classification_report(y_test, baseline_pred))

print("\nLogistic Regression Predictions:")
print(classification_report(y_test, mod.predict(X_test)))

Baseline (Random) Accuracy: 0.48506520824568783

Baseline Classification Report:
              precision    recall  f1-score   support

       False       0.49      0.49      0.49      1196
        True       0.48      0.48      0.48      1181

    accuracy                           0.49      2377
   macro avg       0.49      0.49      0.49      2377
weighted avg       0.49      0.49      0.49      2377


Logistic Regression Predictions:
              precision    recall  f1-score   support

       False       0.59      0.63      0.61      1196
        True       0.60      0.56      0.58      1181

    accuracy                           0.60      2377
   macro avg       0.60      0.60      0.60      2377
weighted avg       0.60      0.60      0.60      2377



In [128]:
TP = [a and b for (a,b) in zip(predictions,y)]
TN = [not a and not b for (a,b) in zip(predictions,y)]
FP = [a and not b for (a,b) in zip(predictions,y)]
FN = [not a and b for (a,b) in zip(predictions,y)]

In [129]:
TP = sum(TP)
TN = sum(TN)
FP = sum(FP)
FN = sum(FN)

In [130]:
BER = 0.5 * (FP / (TN + FP) + FN / (FN + TP))


In [131]:
BER

0.4966509165465467

### n-grams

In [132]:
sentence = "Dark red color, light beige foam"
unigrams = sentence.split()
bigrams = list(zip(unigrams[:-1], unigrams[1:]))
trigrams = list(zip(unigrams[:-2], unigrams[1:-1], unigrams[2:]))

In [133]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))
for d in balanced_merged_data:
    review_text = d.get('review_text', '').lower()  # Default to empty string if key is missing

    # Remove punctuation
    r = ''.join([c for c in review_text if c not in punctuation])

    # Tokenize and filter stopwords
    ws = [w for w in r.split() if w not in stop_words]
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    ws3 = [' '.join(x) for x in list(zip(ws[:-2],ws[1:-1],ws[2:]))]
    ws4 = [' '.join(x) for x in list(zip(ws[:-3],ws[1:-2],ws[2:-1],ws[3:]))]
    ws5 = [' '.join(x) for x in list(zip(ws[:-4],ws[1:-3],ws[2:-2],ws[3:-1],ws[4:]))]
    for w in ws + ws2 + ws3 + ws4 + ws5:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [134]:
len(counts)

872932

In [135]:
words = [x[1] for x in counts[:2000]]

In [136]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [137]:
def feature(datum):
    # Initialize feature vector with zeros
    feat = [0] * len(words)

    # Safely access and preprocess 'review_text'
    review_text = datum.get('review_text', '').lower()
    r = ''.join([c for c in review_text if c not in punctuation])

    # Tokenize and filter out stopwords
    ws = [w for w in r.split() if w not in stop_words]

    # Generate n-grams (1-grams to 5-grams)
    n_grams = []
    for n in range(1, 6):  # 1-grams to 5-grams
        n_grams += [' '.join(ws[i:i + n]) for i in range(len(ws) - n + 1)]

    # Populate feature vector based on n-grams in 'words'
    for w in n_grams:
        if w in words:
            feat[wordId[w]] += 1

    # Add offset for bias term
    feat.append(1)
    return feat

In [138]:
X = [feature(d) for d in balanced_merged_data]
y = [d['recommend'] for d in balanced_merged_data]

In [139]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,shuffle=True)

In [140]:
mod = LogisticRegression(C=1)
mod.fit(X_train, y_train)

predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test
print(sum(correct) / len(correct))

0.7807570977917981


### tf - idf + classifier 

### maybe jacard or cos sim

### data processing:
### grab t

In [141]:
# Count unique users
unique_users = len(set(entry['user_id'] for entry in balanced_merged_data))
print("Number of Unique Users:", unique_users)

# Count unique items
unique_items = len(set(entry['item_id'] for entry in balanced_merged_data))
print("Number of Unique Items:", unique_items)

# Count all items 


Number of Unique Users: 6915
Number of Unique Items: 1738


In [142]:
balanced_merged_data[0]

{'user_id': 'insertnameherere',
 'item_id': '65980',
 'review_text': "less is more?, not really(let me just say that civ 5 was my first civ, so i didn't have any previous civ titles to put it in context, but know when you compard BE to 5 its like its DLC, but also taking things out)i feel like they changed the things that worked, and kept the things i couldn't care for. like the tech web, your pretty much forced to go by the inner ring first, then the second and so forth, it looked more impressive in civ 5 scrolling all the way to the end of the tech tree and seeing how much progress was to be made. The religions gone, which im alright with because it was kinda hard to control and influence anyway, but less is more?The units, while the idea of upgrading them along a specifc line is cool and kind of felt like x-com, the actually outcome of this is that they look slightley different each time and don't really do anything special, you can't have that civ 5 x2 attack, +1 range super units 

In [143]:
unique_user_ids = len(set(entry['user_id'] for entry in balanced_merged_data))
print(f"Number of unique user IDs: {unique_user_ids}")

# To get number of unique item IDs
unique_item_ids = len(set(entry['item_id'] for entry in balanced_merged_data))
print(f"Number of unique item IDs: {unique_item_ids}")

# If you want to see both at once
print("Unique user IDs:", len(set(entry['user_id'] for entry in balanced_merged_data)))
print("Unique item IDs:", len(set(entry['item_id'] for entry in balanced_merged_data)))

Number of unique user IDs: 6915
Number of unique item IDs: 1738
Unique user IDs: 6915
Unique item IDs: 1738


In [144]:
len(balanced_merged_data)

9506

In [145]:
balanced_merged_data_10 = [
    {**entry, 'recommend': 1 if entry['recommend'] else 0} 
    for entry in balanced_merged_data
]

In [146]:
len(balanced_merged_data_10)

9506

In [147]:
merged_data_10 = [
    {**entry, 'recommend': 1 if entry['recommend'] else 0} 
    for entry in merged_data
]

In [148]:
merged_data_10 = merged_data_10[:20000]

In [149]:
len(merged_data_10)

20000

In [150]:
recommendTrain = merged_data_10[:10000]
recommendValid = merged_data_10[10000:]
recommendPerUser = defaultdict(list)
recommendPerItem = defaultdict(list)
for entry in balanced_merged_data_10:
    u = entry['user_id']
    i = entry['item_id']
    r = entry['recommend']  # Now already 0/1
    
    recommendPerUser[u].append((i,r))
    recommendPerItem[i].append((u,r))

In [151]:

# Create a dictionary to track recommendations per item
item_recommendations = defaultdict(lambda: {'total': 0, 'positive': 0})

# Iterate through the balanced_merged_data
for entry in recommendTrain:
    item_id = entry['item_id']
    recommend = entry['recommend']
    
    # Increment total entries for this item
    item_recommendations[item_id]['total'] += 1
    
    # Increment positive recommendations if recommend is 1
    if recommend == 1:
        item_recommendations[item_id]['positive'] += 1

# Find items with 70% or more positive recommendations
highly_recommended_items = [
    item_id for item_id, data in item_recommendations.items() 
    if data['total'] > 0 and (data['positive'] / data['total']) >= 0.70
]

print("Items with 70% or more positive recommendations:", highly_recommended_items)
print("Total number of such items:", len(highly_recommended_items))

Items with 70% or more positive recommendations: ['211420', '72850', '424280', '319630', '730', '12210', '206190', '49520', '39210', '354140', '240', '250320', '227100', '202750', '34900', '70000', '232910', '34330', '8930', '22370', '33230', '346900', '363970', '65790', '57300', '230410', '225840', '6020', '94400', '283940', '283980', '233720', '204880', '218620', '107410', '346110', '4000', '224480', '40800', '211820', '263180', '238460', '300', '208650', '292030', '319910', '271590', '221680', '212070', '304930', '303210', '225540', '261640', '310080', '265930', '8190', '219740', '205100', '620', '252490', '206440', '245550', '232790', '256290', '261030', '105600', '304050', '243870', '238960', '48700', '218230', '394360', '57900', '8870', '113200', '209080', '248570', '204360', '261530', '274900', '286100', '213670', '329950', '223710', '249050', '224060', '254200', '411830', '234670', '2400', '305620', '437220', '317360', '297120', '42910', '348620', '391540', '221100', '386360', 

In [152]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in recommendTrain:
    user,item = d['user_id'], d['item_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = d['recommend']

In [153]:
###Version where only recommended things go into dictionaries 
usersPerItem = defaultdict(set) # Maps an item to the users who recommended it
itemsPerUser = defaultdict(set) # Maps a user to the items they recommended
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in recommendTrain:
    user, item = d['user_id'], d['item_id']
    
    # Only add to sets if recommendation is 1
    if d['recommend'] == 1:
        usersPerItem[item].add(user)
        itemsPerUser[user].add(item)
    
    # Keep ratingDict for all entries
    ratingDict[(user,item)] = d['recommend']

In [154]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for d in recommendTrain:
    user,item = d['user_id'], d['item_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [155]:
##same thing, append only if recommend is 1 (True)
#reviewsPerUser = defaultdict(list)
#reviewsPerItem = defaultdict(list)

for d in recommendTrain:
    user, item = d['user_id'], d['item_id']
    
    # Only append the review if the recommendation is 1
    if d['recommend'] == 1:
        reviewsPerUser[user].append(d)
        reviewsPerItem[item].append(d)

In [156]:
ratingMean = sum([d['recommend'] for d in recommendTrain]) / len(recommendTrain) #0.5 obviously, change later to heuristic

In [157]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [158]:
def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [159]:
def predictRating1(user,item):
    ratings = []
    similarities = []
    maxSim = 0
    for d in reviewsPerUser[user]:
        i2 = d['item_id']
        if i2 == item: continue
        ratings.append(d['recommend'])
        value = CosineSet(usersPerItem[item],usersPerItem[i2])
        maxSim = max(value, maxSim)
        #similarities.append(CosineSet(usersPerItem[item],usersPerItem[i2]))

    #if (sum(similarities) > 0):
        #weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        #return 1 if sum(weightedRatings) / sum(similarities) >= 0.9 else 0
    if maxSim > 0.005:
        return 1
    else:
        return 0#round(ratingMean)

In [160]:
def calculate_accuracy(predictions, labels):
    # Calculate the number of correct predictions
    correct_predictions = sum(1 for pred, label in zip(predictions, labels) if pred == label)
    
    # Calculate accuracy as the percentage of correct predictions
    accuracy = correct_predictions / len(labels)
    
    return accuracy

In [161]:
def predictRating2(user, item):
    ratings = []
    similarities = []
    maxSim = 0
    for d in reviewsPerItem[item]:
        u2 = d['user_id']
        if u2 == user: continue
        ratings.append(d['recommend'])
        value = CosineSet(itemsPerUser[user], itemsPerUser[u2])
        maxSim = max(maxSim,value)
        
    if maxSim > 0.005:
        return 1
        #weightedRatings = [(x*y) for x,y in zip(ratings, similarities)]
        # Round to 0 or 1 for binary classification
        
        #return 1 if sum(weightedRatings) / sum(similarities) >= 0.001 else 0
    else:
        return predictRating1(user,item)

In [162]:
u,i = balanced_merged_data_10[4]['user_id'], balanced_merged_data_10[4]['item_id']
print(predictRating1(u, i))
print(balanced_merged_data_10[4]['user_id'])
print(balanced_merged_data_10[4]['item_id'])
print(balanced_merged_data_10[4])

0
bindisposerer
227940
{'user_id': 'bindisposerer', 'item_id': '227940', 'review_text': 'This game is essentially Battlefield 1943 on Pay 2 Win.Want a better game? http://battlefield1943.com/content/game', 'recommend': 0, 'playtime_forever': 37, 'playtime_2weeks': 0}


In [163]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [164]:
alwaysPredictMean = [ratingMean for d in recommendValid]

In [165]:
simPredictions = [predictRating2(d['user_id'], d['item_id']) for d in recommendValid]

In [166]:
labels = [d['recommend'] for d in recommendValid]

In [167]:
MSE(alwaysPredictMean, labels)

0.10504036000000001

In [168]:
MSE(simPredictions, labels)

0.8808

In [169]:
# Always predict mean baseline (rounded to 0 or 1)
#alwaysPredictMean = [1 if ratingMean >= 0.5 else 0 for d in recommendValid]
alwaysPredictMean = [random.randint(0, 1) for d in recommendValid]

# Get predictions using the modified predictRating2
simPredictions = [predictRating2(d['user_id'], d['item_id']) for d in recommendValid]

# Get true labels
labels = [d['recommend'] for d in recommendValid]

# Calculate accuracies
mean_baseline_accuracy = calculate_accuracy(alwaysPredictMean, labels)
similarity_based_accuracy = calculate_accuracy(simPredictions, labels)

print("Mean Baseline Accuracy:", mean_baseline_accuracy)
print("Similarity-based Prediction Accuracy:", similarity_based_accuracy)

Mean Baseline Accuracy: 0.5023
Similarity-based Prediction Accuracy: 0.1192


In [170]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

def prepare_surprise_dataset(combined_data):
    # Create a custom reader to handle the data format
    reader = Reader(rating_scale=(0, 1))
    
    # Convert data to Surprise format
    ratings_dict = {
        'user': [entry['user_id'] for entry in combined_data],
        'item': [entry['item_id'] for entry in combined_data],
        'rating': [1 if entry['recommend'] else 0 for entry in combined_data]
    }
    
    df = pd.DataFrame(ratings_dict)
    surprise_dataset = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)
    
    return surprise_dataset

def train_recommendation_model(surprise_dataset):
    # Split the dataset
    trainset, testset = train_test_split(surprise_dataset, test_size=0.2)
    
    # Use SVD algorithm
    model = SVD(
        n_factors=100,  # Reduce dimensionality
        n_epochs=20,    # Limit training iterations
        lr_all=0.005,   # Learning rate
        reg_all=0.02    # Regularization to prevent overfitting
    )
    
    # Train the model
    model.fit(trainset)
    
    # Evaluate
    predictions = model.test(testset)
    print("RMSE:", accuracy.rmse(predictions))
    print("MAE:", accuracy.mae(predictions))
    
    return model

# Usage
surprise_dataset = prepare_surprise_dataset(balanced_merged_data)
recommendation_model = train_recommendation_model(surprise_dataset)

# Prediction function
def predict_recommendation(model, user_id, item_id):
    # Predict recommendation probability
    prediction = model.predict(user_id, item_id)
    return prediction.est  # Estimated rating/recommendation probability

RMSE: 0.4610
RMSE: 0.4610251817494337
MAE:  0.4291
MAE: 0.4290654374769219


In [171]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix,
    classification_report
)

def prepare_surprise_dataset(combined_data):
    # Create a custom reader to handle the data format
    reader = Reader(rating_scale=(0, 1))
    
    # Convert data to Surprise format
    ratings_dict = {
        'user': [entry['user_id'] for entry in combined_data],
        'item': [entry['item_id'] for entry in combined_data],
        'rating': [1 if entry['recommend'] else 0 for entry in combined_data]
    }
    
    df = pd.DataFrame(ratings_dict)
    surprise_dataset = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)
    
    return surprise_dataset

def train_and_evaluate_recommendation_model(combined_data):
    # Prepare dataset
    surprise_dataset = prepare_surprise_dataset(combined_data)
    
    # Split the dataset
    trainset, testset = train_test_split(surprise_dataset, test_size=0.2)
    
    # Use SVD algorithm
    model = SVD(
        n_factors=100,  # Reduce dimensionality
        n_epochs=20,    # Limit training iterations
        lr_all=0.005,   # Learning rate
        reg_all=0.02    # Regularization to prevent overfitting
    )
    
    # Train the model
    model.fit(trainset)
    
    # Predict on test set
    predictions = model.test(testset)
    
    # Convert predictions to binary (0 or 1)
    y_true = [pred.r_ui for pred in predictions]
    y_pred = [1 if pred.est >= 0.515 else 0 for pred in predictions]
    
    # Calculate metrics
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    
    # Detailed metrics
    print("\nDetailed Metrics:")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(cm)
    
    return model

# Usage
model = train_and_evaluate_recommendation_model(balanced_merged_data)

# Prediction function
def predict_recommendation(model, user_id, item_id):
    # Predict recommendation probability
    prediction = model.predict(user_id, item_id)
    return prediction.est  # Estimated rating/recommendation probability

Classification Report:
              precision    recall  f1-score   support

         0.0       0.71      0.69      0.70       996
         1.0       0.67      0.69      0.68       906

    accuracy                           0.69      1902
   macro avg       0.69      0.69      0.69      1902
weighted avg       0.69      0.69      0.69      1902


Detailed Metrics:
Accuracy: 0.6898002103049422
Precision: 0.6688034188034188
Recall: 0.6909492273730684
F1 Score: 0.6796959826275787

Confusion Matrix:
[[686 310]
 [280 626]]


### latent factors (svs or matrix factorization)

In [172]:
len(cleaned_item_metadata_further)

1189

In [173]:
len(balanced_merged_data) 

9506

In [174]:
cleaned_item_metadata_further[14]

{'publisher': 'Egosoft',
 'genres': ['Action', 'Simulation', 'Strategy'],
 'app_name': 'X3: Terran Conflict',
 'sentiment': 'Very Positive',
 'title': 'X3: Terran Conflict',
 'url': 'http://store.steampowered.com/app/2820/X3_Terran_Conflict/',
 'release_date': '2008-10-16',
 'tags': ['Space',
  'Simulation',
  'Sci-fi',
  'Sandbox',
  'Strategy',
  'Space Sim',
  'Open World',
  'Trading',
  'Singleplayer',
  'Economy',
  'Action',
  'Capitalism',
  'Great Soundtrack',
  'Moddable',
  '4X',
  'Flight',
  'Atmospheric',
  'Multiplayer'],
 'reviews_url': 'http://steamcommunity.com/app/2820/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player',
  'Steam Achievements',
  'Full controller support',
  'Captions available',
  'Steam Cloud'],
 'price': 15.99,
 'metascore': 73,
 'early_access': False,
 'id': '2820',
 'developer': 'Egosoft'}

In [175]:
# Step 1: Create a set of 'id' values from cleaned_item_metadata_further for fast lookup
item_metadata_ids = {entry['id'] for entry in cleaned_item_metadata_further}

# Step 2: Create a new list to hold the combined entries
combined_data = []

# Step 3: Iterate through balanced_merged_data and combine with metadata
for entry in balanced_merged_data:
    item_id = entry['item_id']
    
    # Step 4: If the item_id exists in cleaned_item_metadata_further, combine data
    if item_id in item_metadata_ids:
        # Find the corresponding metadata entry in cleaned_item_metadata_further
        metadata_entry = next(item for item in cleaned_item_metadata_further if item['id'] == item_id)
        
        # Combine the metadata with the entry from balanced_merged_data
        combined_entry = {
            **entry,  # Include all fields from balanced_merged_data entry
            'publisher': metadata_entry.get('publisher'),
            'genres': metadata_entry.get('genres'),
            'tags': metadata_entry.get('tags'),
            'price': metadata_entry.get('price'),
            'metascore':metadata_entry.get('metascore'),
            'developer': metadata_entry.get('developer'),
            'sentiment': metadata_entry.get('sentiment')
        }
        
        # Add the combined entry to the new list
        combined_data.append(combined_entry)

# Step 5: Optionally, print the number of combined entries
print(f"Combined data contains {len(combined_data)} entries.")

Combined data contains 6839 entries.


In [176]:
combined_data[0]

{'user_id': 'Dante69123',
 'item_id': '58610',
 'review_text': "If you were expecting a first person shooter it's not. This game gave me cancer",
 'recommend': False,
 'playtime_forever': 31,
 'playtime_2weeks': 0,
 'publisher': 'Focus Home Interactive',
 'genres': ['Strategy'],
 'tags': ['Strategy',
  'RTS',
  'Military',
  'Cold War',
  'Wargame',
  'Realistic',
  'Real Time Tactics',
  'Alternate History',
  'War',
  'Action',
  'Simulation',
  'Singleplayer',
  'Multiplayer',
  'Real-Time',
  'Tactical'],
 'price': 9.99,
 'metascore': 81,
 'developer': 'Eugen Systems',
 'sentiment': 'Mostly Positive'}

In [177]:
# Step 1: Create a set of (user_id, item_id) pairs from balanced_merged_data
balanced_merged_data_pairs = {
    (entry['user_id'], entry['item_id']) for entry in balanced_merged_data
}

# Step 2: Check if each (user_id, item_id) in combined_data exists in balanced_merged_data_pairs
for entry in combined_data:
    user_id = entry['user_id']
    item_id = entry['item_id']
    
    # If the pair (user_id, item_id) is not found in balanced_merged_data_pairs
    if (user_id, item_id) not in balanced_merged_data_pairs:
        print(f"Mismatch found: user_id={user_id}, item_id={item_id}")
        # You can also raise an error here if needed, for example:
        # raise ValueError(f"Pair not found: user_id={user_id}, item_id={item_id}")

# Step 3: If no mismatches are found, print a confirmation message
print("All pairs in combined_data are present in balanced_merged_data.")

All pairs in combined_data are present in balanced_merged_data.


In [178]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Step 1: Combine genres and tags into a single string for each game
combined_data_with_text = []
for review in combined_data:
    genres_text = " ".join(review['genres']) if review['genres'] else ""
    tags_text = " ".join(review['tags']) if review['tags'] else ""
    combined_data_with_text.append(genres_text + " " + tags_text)


### one hot encodings of genres and tags

In [179]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack, csr_matrix

In [180]:
len(combined_data)

6839

In [181]:
combined_data

[{'user_id': 'Dante69123',
  'item_id': '58610',
  'review_text': "If you were expecting a first person shooter it's not. This game gave me cancer",
  'recommend': False,
  'playtime_forever': 31,
  'playtime_2weeks': 0,
  'publisher': 'Focus Home Interactive',
  'genres': ['Strategy'],
  'tags': ['Strategy',
   'RTS',
   'Military',
   'Cold War',
   'Wargame',
   'Realistic',
   'Real Time Tactics',
   'Alternate History',
   'War',
   'Action',
   'Simulation',
   'Singleplayer',
   'Multiplayer',
   'Real-Time',
   'Tactical'],
  'price': 9.99,
  'metascore': 81,
  'developer': 'Eugen Systems',
  'sentiment': 'Mostly Positive'},
 {'user_id': 'bindisposerer',
  'item_id': '227940',
  'review_text': 'This game is essentially Battlefield 1943 on Pay 2 Win.Want a better game? http://battlefield1943.com/content/game',
  'recommend': False,
  'playtime_forever': 37,
  'playtime_2weeks': 0,
  'publisher': 'Reto-Moto',
  'genres': ['Action', 'Free to Play', 'Indie', 'Massively Multiplayer'

In [182]:
def prepare_features(combined_data, max_features=1000):
    # Create a DataFrame
    df = pd.DataFrame(combined_data)
    
    # Quick price conversion
    #df['price'] = pd.to_numeric(df['price'].replace('Free to Play', 0), errors='coerce').fillna(0)
    
    # Faster list handling
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    df['tags'] = df['tags'].apply(lambda x: x if isinstance(x, list) else [])
    
    # More efficient MultiLabelBinarizer
    mlb_genres = MultiLabelBinarizer(sparse_output=False)
    mlb_tags = MultiLabelBinarizer(sparse_output=True)
    
    # Sparse one-hot encoding
    genres_encoded = mlb_genres.fit_transform(df['genres'])
    tags_encoded = mlb_tags.fit_transform(df['tags'])
    
    # Sparse feature matrix construction
    #price_col = csr_matrix(df['price'].values.reshape(-1, 1))
    #playtime_col = csr_matrix(df['playtime_forever'].values.reshape(-1, 1))
    
    # Combine features sparsely
    X = hstack([
        #price_col, 
        #playtime_col, 
        genres_encoded[:, :max_features], 
        tags_encoded[:, :max_features]
    ])
    
    # Prepare target
    y = df['recommend'].astype(int)
    
    # Collect feature names for later use
    feature_names = list(mlb_genres.classes_[:max_features]) + list(mlb_tags.classes_[:max_features])
    
    return X, y, feature_names

def train_and_evaluate(X, y, feature_names):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    
    # Train with fewer iterations and stronger regularization
    model = LogisticRegression(max_iter=1200, C=1)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Show important features (sorted by absolute value of coefficients)
    importance = np.abs(model.coef_).flatten()
    sorted_idx = np.argsort(importance)[::-1]  # Sort in descending order
    
    print("\nTop important features:")
    for idx in sorted_idx[:40]:  # Show top 10 important features
        print(f"{feature_names[idx]}: {importance[idx]:.4f}")
    
    return model

# Main execution
X, y, feature_names = prepare_features(combined_data)
#convert to 1/0 replacing recomment true/false with 1/0
model = train_and_evaluate(X, y, feature_names)

Accuracy: 0.6684210526315789

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.61      0.63       790
           1       0.68      0.72      0.70       920

    accuracy                           0.67      1710
   macro avg       0.67      0.66      0.66      1710
weighted avg       0.67      0.67      0.67      1710


Top important features:
Science: 2.0824
Based On A Novel: 1.3400
Turn-Based Tactics: 1.3217
Inventory Management: 1.2757
Animation & Modeling: 1.2624
Satire: 1.2448
Dungeon Crawler: 1.2199
Documentary: 1.1864
Villain Protagonist: 1.1718
Interactive Fiction: 1.1578
RPGMaker: 1.1559
Agriculture: 1.1449
1980s: 1.1080
Character Customization: 1.0734
Lara Croft: 1.0331
Futuristic: 1.0164
Utilities: 1.0122
Utilities: 1.0122
2.5D: 0.9788
Drama: 0.9521
Gun Customization: 0.9487
Twin Stick Shooter: 0.9234
Underwater: 0.9134
Turn-Based Strategy: 0.9094
Flight: 0.9017
Psychological: 0.8999
Romance: 0.8884
Benchmark: 0.8826

In [191]:
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack, csr_matrix
import pandas as pd
import numpy as np


def prepare_features(combined_data, max_features=1500):
    # Create a DataFrame
    df = pd.DataFrame(combined_data)
    
    # Replace NaN in metascore with the global average
    global_avg_metascore = df['metascore'].apply(pd.to_numeric, errors='coerce').mean(skipna=True)
    df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce').fillna(global_avg_metascore)
    metascore_col = csr_matrix(
        MinMaxScaler().fit_transform(df['metascore'].values.reshape(-1, 1))
    )
    
    # One-hot encode genres and tags
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    df['tags'] = df['tags'].apply(lambda x: x if isinstance(x, list) else [])
    
    mlb_genres = MultiLabelBinarizer(sparse_output=True)
    mlb_tags = MultiLabelBinarizer(sparse_output=True)
    
    genres_encoded = mlb_genres.fit_transform(df['genres'])
    tags_encoded = mlb_tags.fit_transform(df['tags'])
    
    # One-hot encode developer and publisher
    lb_developer = LabelBinarizer(sparse_output=True)
    lb_publisher = LabelBinarizer(sparse_output=True)
    
    developer_encoded = lb_developer.fit_transform(df['developer'].fillna('Unknown'))
    publisher_encoded = lb_publisher.fit_transform(df['publisher'].fillna('Unknown'))
    
    # Combine features sparsely
    X = hstack([
        metascore_col,
        genres_encoded[:, :max_features],
        tags_encoded[:, :max_features],
        developer_encoded,
        #publisher_encoded
    ])
    
    # Prepare target
    y = df['recommend'].astype(int)
    
    # Collect feature names
    feature_names = (
        ['metascore'] +
        list(mlb_genres.classes_[:max_features]) +
        list(mlb_tags.classes_[:max_features]) +
        list(lb_developer.classes_) +
        list(lb_publisher.classes_)
    )
    
    return X, y, feature_names


def train_and_evaluate(X, y, feature_names):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
    # Define classifiers
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1200, C=1),
        #"Naive Bayes": MultinomialNB(),
        #"Linear SVM": LinearSVC(C=1, max_iter=2000),
        #"Ridge Classifier": RidgeClassifier(alpha=1.5),
        #"SGD Classifier (Log)": SGDClassifier(loss='log_loss', max_iter=1000, alpha=0.001),
    }
    
    # Evaluate each classifier
    for name, model in models.items():
        print(f"\n--- {name} ---")
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        
        # Metrics
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        # Show feature importance for Logistic Regression
        if name == "Logistic Regression":
            importance = np.abs(model.coef_).flatten()
            sorted_idx = np.argsort(importance)[::-1]  # Sort in descending order
            
            print("\nTop important features:")
            for idx in sorted_idx[:40]:  # Show top 40 important features
                print(f"{feature_names[idx]}: {importance[idx]:.4f}")
    
    return models


# Main execution
X, y, feature_names = prepare_features(combined_data)
models = train_and_evaluate(X, y, feature_names)


--- Logistic Regression ---
Accuracy: 0.6995614035087719

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.63      0.66       645
           1       0.70      0.76      0.73       723

    accuracy                           0.70      1368
   macro avg       0.70      0.70      0.70      1368
weighted avg       0.70      0.70      0.70      1368


Top important features:
metascore: 2.0976
Spiderling Studios: 1.9604
Rockstar North: 1.4803
Eko Software: 1.4048
Rocksteady Studios: 1.3721
Nina Freeman,Emmett Butler,Decky Coss,Joni Kittaka: 1.2840
Science: 1.2583
Animation & Modeling: 1.2274
Exe Games Inc.: 1.2120
Satire: 1.2022
X-Legend: 1.1785
Free Lives: 1.1757
FIX Korea, Co.LTD,FIX Games, Co. LTD: 1.1716
Dark Fantasy: 1.1523
Gaijin Games: 1.1065
Beat 'em up: 1.1046
Gun Customization: 1.0993
Techland: 1.0458
Trading: 1.0340
Hunting: 1.0319
EA Redwood Shores: 1.0165
Wyrmbyte: 1.0077
Doctor Entertainment AB: 0.9981
CCP: 0.9911
Dra

In [184]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import pandas as pd
import numpy as np

def prepare_features(combined_data, max_features=1000):
    # Create a DataFrame
    df = pd.DataFrame(combined_data)
    
    # Replace NaN in metascore with the global average
    global_avg_metascore = df['metascore'].apply(pd.to_numeric, errors='coerce').mean(skipna=True)
    df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce').fillna(global_avg_metascore)
    metascore_col = csr_matrix(
        MinMaxScaler().fit_transform(df['metascore'].values.reshape(-1, 1))
    )
    
    # One-hot encode genres and tags
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    df['tags'] = df['tags'].apply(lambda x: x if isinstance(x, list) else [])
    
    mlb_genres = MultiLabelBinarizer(sparse_output=True)
    mlb_tags = MultiLabelBinarizer(sparse_output=True)
    
    genres_encoded = mlb_genres.fit_transform(df['genres'])
    tags_encoded = mlb_tags.fit_transform(df['tags'])
    
    # Label encode developer, publisher, and sentiment as multi-class categorical variables
    le_developer = LabelEncoder()
    le_publisher = LabelEncoder()
    le_sentiment = LabelEncoder()
    
    # Handling missing values for developer, publisher, and sentiment
    df['developer'] = le_developer.fit_transform(df['developer'].fillna('Unknown'))
    df['publisher'] = le_publisher.fit_transform(df['publisher'].fillna('Unknown'))
    df['sentiment'] = le_sentiment.fit_transform(df['sentiment'].fillna('Unknown'))  # Label encoding sentiment
    
    # Combine features sparsely
    X = hstack([
        metascore_col,
        genres_encoded[:, :max_features],
        tags_encoded[:, :max_features],
        csr_matrix(df['developer'].values.reshape(-1, 1)),
        csr_matrix(df['publisher'].values.reshape(-1, 1)),
        csr_matrix(df['sentiment'].values.reshape(-1, 1))  # Add sentiment as a feature
    ])
    
    # Prepare target
    y = df['recommend'].astype(int)
    
    # Collect feature names
    feature_names = (
        ['metascore'] +
        list(mlb_genres.classes_[:max_features]) +
        list(mlb_tags.classes_[:max_features]) +
        ['developer'] + 
        ['publisher'] +
        ['sentiment']  # Add sentiment feature name
    )
    
    return X, y, feature_names

def train_and_evaluate(X, y, feature_names):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
    # Define classifiers
    models = {
        "Logistic Regression": LogisticRegression(max_iter=2000, C=1.3),
        #"Naive Bayes": MultinomialNB(),
        #"Linear SVM": LinearSVC(C=1.3, max_iter=2000),
        #"Ridge Classifier": RidgeClassifier(alpha=0.5),
        #"SGD Classifier (Log)": SGDClassifier(loss='log_loss', max_iter=1000, alpha=0.001),
    }
    
    # Evaluate each classifier
    for name, model in models.items():
        print(f"\n--- {name} ---")
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        
        # Metrics
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        # Show feature importance for Logistic Regression
        if name == "Logistic Regression":
            importance = np.abs(model.coef_).flatten()
            sorted_idx = np.argsort(importance)[::-1]  # Sort in descending order
            
            print("\nTop important features:")
            for idx in sorted_idx[:40]:  # Show top 40 important features
                print(f"{feature_names[idx]}: {importance[idx]:.4f}")
    
    return models


# Main execution
X, y, feature_names = prepare_features(combined_data)
models = train_and_evaluate(X, y, feature_names)


--- Logistic Regression ---
Accuracy: 0.6915204678362573

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.62      0.66       646
           1       0.69      0.75      0.72       722

    accuracy                           0.69      1368
   macro avg       0.69      0.69      0.69      1368
weighted avg       0.69      0.69      0.69      1368


Top important features:
metascore: 2.0914
Gun Customization: 1.4237
1980s: 1.4142
Science: 1.2484
Satire: 1.1575
Trading: 1.1572
Beat 'em up: 1.0649
Dinosaurs: 1.0296
Twin Stick Shooter: 0.9858
GameMaker: 0.9548
4 Player Local: 0.9314
Underwater: 0.9312
Dark Fantasy: 0.9040
City Builder: 0.8630
4X: 0.8607
Puzzle-Platformer: 0.8486
Tower Defense: 0.8333
Gambling: 0.8286
Level Editor: 0.8056
Modern: 0.7971
Turn-Based: 0.7877
Platformer: 0.7859
Tanks: 0.7777
Animation & Modeling: 0.7774
RTS: 0.7773
Dungeon Crawler: 0.7713
Fighting: 0.7693
Minimalist: 0.7613
Romance: 0.7574
Turn-Based St

c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
